##Importing necessary libraries & packages

In [ ]:
import pandas as pd
import json
import os
import hashlib
import re
!pip install langdetect
from langdetect import detect, LangDetectException
from datetime import datetime
import time
import nltk
from nltk.tokenize import word_tokenize
import pytz
import unicodedata
import random

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 12.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993223 sha256=8a8dd8f719fcdaa18bceaaeea6fd46864da690e6876f5628c3784b6869ddf4b8
  Stored in directory: /root/.cache/pip/wheels/0a/f2/b2/e5ca405801e05eb7c8ed5b3b4bcf1fcabcd6272c167640072e
Successfully built langdetect


# Reddit data retreaval

In [ ]:
#installing the necessary packages to use the Reddit API
!pip install praw
import praw

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.3/189.3 kB 4.4 MB/s eta 0:00:00


In [ ]:
#unloading my Reddit password from the text file to save as 'reddit_password'
with open('/content/Password.1.txt', 'r', encoding='utf-8-sig') as f:
    reddit_password = f.read().strip()

In [ ]:
#authorising the Reddit API using the client secret and client ID I obtained when setting up my script app, as well as my username and password
reddit = praw.Reddit(
    client_secret="p-54bpBMHdRIbmbyzMTRFS0ByoudEg",
    client_id="DLpyGZbv-atA_MEOKVUPCQ",
    user_agent="My API Capstone/v3",
    username="Capstone2025",
    password=reddit_password
)

In [ ]:
#the list of subreddits I manually identified as having ULEZ discourse
subreddits = [
    "OutOfTheLoop", "CarTalkUK", "VanLifeUK", "Britain", "drivingUK", "London", "carmemes",
    "LegalAdviceUK", "fuckcars", "britishproblems", "unitedkingdom", "Conservative", "transit",
    "Miata", "LandroverDefender", "ukpolitics", "TeslaUK", "MotoUK", "AskUK", "brighton",
    "northernireland", "Asmongold", "Edinburgh", "chessington", "GolfGTI", "mr2", "BMWE36",
    "VFR", "Cyberpunk", "compoface", "walkaway", "ukraine", "classicminis", "okmatewanker",
    "TheRestIsPolitics", "TransportForLondon", "LegalAdviceEurope", "AmazonFlexUK", "croydon",
    "londoncycling", "science", "waze", "bestconspiracymemes", "ConspiracyMemesII", "uknews",
    "e46", "AskLondon", "chaoticgood", "LabourUK", "europe"
]

After creating an empty list, I iterate through the subreddits previously defined, for each one, I search for any post with 'ULEZ' in the title. Following this, I convert this current UNIX time into a readable datetime. Using this I filter out posts before 2019. I then extract and merge the title and body text of the post. Following this, I also retrieve the authors, comments, date, and subreddit. After converting the times into readable formats, I append everything into the list ulez_reddit. Finally, I convert this into a dataframe and sort everything by date.



In [ ]:
ulez_reddit = []

for subreddit in subreddits:
    for post in reddit.subreddit(subreddit).search("title:ulez", sort="top", time_filter="all", limit=100):
        post_date = datetime.utcfromtimestamp(post.created_utc)
        if post_date.year < 2019:
            continue
        full_text = (post.title or "") + "\n\n" + (post.selftext or "")
        ulez_reddit.append({
            "from": str(post.author),
            "message": full_text,
            "date": post_date,
            "subreddit": subreddit
        })
        post.comments.replace_more(limit=None)
        for comment in post.comments.list():
            comment_date = datetime.utcfromtimestamp(comment.created_utc)
            ulez_reddit.append({
                "from": str(comment.author),
                "message": comment.body,
                "date": comment_date,
                "subreddit": subreddit
            })

reddit_ulez = pd.DataFrame(ulez_reddit)
reddit_ulez = reddit_ulez.sort_values(by="date")

Streaming output truncated to the last 5000 lines.
It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.r

# Converting Telegram data to Dataframe

The extract_text() function is used to extract only the readable text from Telegram's JSON message structure. Telegram often stores text in a list of dictionaries, where each dictionary includes formatting or message type info (e.g., "plain" or "link"). This function removes that extra formatting and joins everything into a clean string of plain text.

The jsonmerge_from_folder() function reads and merges all the Telegram JSON files stored in the 'Capstone data' folder on Google Colab. For each file, it opens the JSON, extracts the messages and group name, and loops through each message. If the message is a standard text message, it uses extract_text() to get clean text. It also collects other details like the date, sender ID, and the group title. All of this is saved into a single pandas DataFrame for analysis.

In [ ]:
def extract_text(message):
    if isinstance(message, list):
        return ''.join(
            part.get('text', str(part)) if isinstance(part, dict) else str(part)
            for part in message
        )
    return str(message)

def jsonmerge_from_folder(folder_path):
    telegram_ulez = []
    json_files = [f for f in os.listdir(folder_path) if f.endswith('.json')]
    for i, filename in enumerate(json_files, 1):
        with open(os.path.join(folder_path, filename), 'r', encoding='utf-8') as f:
            data = json.load(f)
            messages = data.get('messages', [])
            group_title = data.get('name', f'Group_{i}')
            for msg in messages:
                if msg.get('type') == 'message':
                    telegram_ulez.append({
                        'date': msg.get('date'),
                        'from': msg.get('from_id', 'Unknown'),
                        'message': extract_text(msg.get('text', '')),
                        'group_name': group_title,
                    })

    return pd.DataFrame(telegram_ulez)

In [ ]:
telegram_ulez = jsonmerge_from_folder('/content/Telegram ULEZ data')

#Pre-processing the data

The function below uses the SHA512 hash generator to replace usernames with a unique cryptographic hash. This allows me to keep track of the user while removing personally identifiable information. This allows for pattern analysis (e.g., identifying key disseminators of misinformation) while protecting individual privacy. The original usernames are replaced with hashed versions and then dropped from the dataset.

In [ ]:
def hash_username(username):
    hashGen = hashlib.sha512()
    hashGen.update(str(username).encode('utf-8'))
    return hashGen.hexdigest()

In [ ]:
telegram_ulez['hashed_user'] = telegram_ulez['from'].apply(hash_username)
telegram_ulez = telegram_ulez.drop('from', axis=1)

reddit_ulez['hashed_user'] = reddit_ulez['from'].apply(hash_username)
reddit_ulez = reddit_ulez.drop('from', axis=1)

In [ ]:
telegram_ulez

,date,group_name,hashed_user,clean_message
6,2023-05-22T08:35:00,"Action Against ULEZ, CAZ, LTNs & 15-20 minute ...",420dd7cabeeb97d24d5a16dfc5cc9af7e247ea1f7a0703...,old article but !
10,2023-05-25T08:37:57,"Action Against ULEZ, CAZ, LTNs & 15-20 minute ...",420dd7cabeeb97d24d5a16dfc5cc9af7e247ea1f7a0703...,anyone aware of this great rising organisers m...
11,2023-05-25T08:39:42,"Action Against ULEZ, CAZ, LTNs & 15-20 minute ...",bd1383bdf1e89ee4484adecd58b3b78c3c1fe136427e12...,don’t know anything about it
21,2023-06-14T07:21:00,"Action Against ULEZ, CAZ, LTNs & 15-20 minute ...",420dd7cabeeb97d24d5a16dfc5cc9af7e247ea1f7a0703...,"new - un claims ""misinformation, disinformatio..."
26,2023-06-14T08:04:31,"Action Against ULEZ, CAZ, LTNs & 15-20 minute ...",bd1383bdf1e89ee4484adecd58b3b78c3c1fe136427e12...,i have to say that the one thing i’m most conc...
...,...,...,...,...
47384,2021-04-21T16:47:35,Newham People Power,4c6ea74f64650c0bb72a0415a857aab21b36b0479b6e27...,time for a change. newham referendum.vote comm...
47386,2021-05-05T14:05:18,Newham People Power,4c6ea74f64650c0bb72a0415a857aab21b36b0479b6e27...,"1 more day to go, please share with everyone. ..."
47387,2021-09-03T10:45:50,Newham People Power,4c6ea74f64650c0bb72a0415a857aab21b36b0479b6e27...,"*mayor rokhsana fiaz, bring back yellow visito..."
47388,2021-09-08T11:26:33,Newham People Power,4c6ea74f64650c0bb72a0415a857aab21b36b0479b6e27...,please share with everyone.if there are any ca...


The function, clean_text, cleans text by decoding weird formatting which sometimes happens with scraped data. It then replaces mentions with @removed to protect anonymity, removes links, removes trailing whitespace, and removes emojis.  Following this, I use the Unicode category in Python’s unicodedata module to filter out unwanted characters, keeping only readable characters: letters, numbers, punctuation, spaces, and symbols. Finally, everything was put into lowercase.

In [ ]:
def clean_text(text):
    if pd.isnull(text):
        return text
    try:
        text = text.encode('latin1').decode('utf-8')
    except (UnicodeEncodeError, UnicodeDecodeError):
        pass
    text = re.sub(r'@\w+', '@removed', text)
    text = re.sub(r'http[s]?://\S+|www\.\S+|t\.me/\S+|dlive\.tv/\S+', '', text)
    emoji_pattern = re.compile(
        "["
        u"\U0001F600-\U0001F64F"
        u"\U0001F300-\U0001F5FF"
        u"\U0001F680-\U0001F6FF"
        u"\U0001F700-\U0001F77F"
        u"\U0001F780-\U0001F7FF"
        u"\U0001F800-\U0001F8FF"
        u"\U0001F900-\U0001F9FF"
        u"\U0001FA00-\U0001FA6F"
        u"\U0001FA70-\U0001FAFF"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        "]+", flags=re.UNICODE)
    text = emoji_pattern.sub(r'', text)
    text = ''.join(
        c for c in text
        if unicodedata.category(c)[0] in ['L', 'N', 'P', 'Z', 'S']
    )
    text = re.sub(r'\s+', ' ', text).strip().lower()
    return text

The function below uses the langdetect library to check whether the text is written in English. After that, all messages are processed to remove links. Then, any messages that are either empty (after link removal) or not written in English are filtered out and dropped.

In [ ]:
def is_english(text):
    try:
        return detect(str(text)) == 'en'
    except LangDetectException:
        return False

In [ ]:
#applying functions to the data and dropping the message column, which isn't clean
telegram_ulez['clean_message'] = telegram_ulez['message'].apply(clean_text)
telegram_ulez = telegram_ulez[telegram_ulez['clean_message'].apply(is_english)]
telegram_ulez = telegram_ulez.drop('message', axis=1)

reddit_ulez['clean_message'] = reddit_ulez['message'].apply(clean_text)
reddit_ulez = reddit_ulez[reddit_ulez['clean_message'].apply(is_english)]
reddit_ulez = reddit_ulez.drop('message', axis=1)

In [ ]:
#converting dataframes to CSVs and saving them to my files
from google.colab import files
telegram_ulez.to_csv('final_clean_telegram_ulez.csv', index=False)
files.download('final_clean_telegram_ulez.csv')

reddit_ulez.to_csv('final_clean_reddit_ulez.csv', index=False)
files.download('final_clean_reddit_ulez.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
telegram_ulez

,date,group_name,hashed_user,clean_message
6,2023-05-22T08:35:00,"Action Against ULEZ, CAZ, LTNs & 15-20 minute ...",420dd7cabeeb97d24d5a16dfc5cc9af7e247ea1f7a0703...,old article but !
10,2023-05-25T08:37:57,"Action Against ULEZ, CAZ, LTNs & 15-20 minute ...",420dd7cabeeb97d24d5a16dfc5cc9af7e247ea1f7a0703...,anyone aware of this great rising organisers m...
11,2023-05-25T08:39:42,"Action Against ULEZ, CAZ, LTNs & 15-20 minute ...",bd1383bdf1e89ee4484adecd58b3b78c3c1fe136427e12...,don’t know anything about it
21,2023-06-14T07:21:00,"Action Against ULEZ, CAZ, LTNs & 15-20 minute ...",420dd7cabeeb97d24d5a16dfc5cc9af7e247ea1f7a0703...,"new - un claims ""misinformation, disinformatio..."
26,2023-06-14T08:04:31,"Action Against ULEZ, CAZ, LTNs & 15-20 minute ...",bd1383bdf1e89ee4484adecd58b3b78c3c1fe136427e12...,i have to say that the one thing i’m most conc...
...,...,...,...,...
47384,2021-04-21T16:47:35,Newham People Power,4c6ea74f64650c0bb72a0415a857aab21b36b0479b6e27...,time for a change. newham referendum.vote comm...
47386,2021-05-05T14:05:18,Newham People Power,4c6ea74f64650c0bb72a0415a857aab21b36b0479b6e27...,"1 more day to go, please share with everyone. ..."
47387,2021-09-03T10:45:50,Newham People Power,4c6ea74f64650c0bb72a0415a857aab21b36b0479b6e27...,"*mayor rokhsana fiaz, bring back yellow visito..."
47388,2021-09-08T11:26:33,Newham People Power,4c6ea74f64650c0bb72a0415a857aab21b36b0479b6e27...,please share with everyone.if there are any ca...


# Scrambling the data for the Capstone appendix

In [ ]:
telegram = pd.read_csv('/content/clean_telegram_ulez.csv')
reddit = pd.read_csv('/content/clean_reddit_ulez.csv')

In [ ]:
#the function below tokenises words and randomly shuffles them to ensure they can't be traced back to a user for the capstone appendix
def scramble_words(message):
    words = word_tokenize(message)
    random.shuffle(words)
    return ' '.join(words)

In [ ]:
telegram['scrambled'] = telegram['clean_message'].apply(scramble_words)
telegram = telegram.drop('clean_message', axis=1)

reddit['scrambled'] = reddit['clean_message'].apply(scramble_words)
reddit = reddit.drop('clean_message', axis=1)

In [ ]:
telegram.to_csv('scrambled_telegram_ulez.csv', index=False)
files.download('scrambled_telegram_ulez.csv')

reddit.to_csv('scrambled_reddit_ulez.csv', index=False)
files.download('scrambled_reddit_ulez.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>